# Imaginary Power from Time-Offset Visibilities

Bobby Pascua and the HERA Validation Team

## Setup and Metadata

### Abstract

In this notebook, we show how a power spectrum estimate may have a nonvanishing imaginary component when derived from time-offset visibility measurements of an uncorrelated, Gaussian sky.

### Imports

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import multiprocessing as mp
from IPython.utils import io
from datetime import datetime

import aipy
import healpy as hpy
import hera_pspec as hp
from pyuvdata import UVData
import os
import glob

In [2]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

<IPython.core.display.Javascript object>

### Description

Last executed: {str(datetime.now())}

- **Major Step Description:** Simulate sky-locked, white-spectrum Gaussian field and estimate power spectrum with `hera_pspec.pspecdata`.
- **Minor Variation Description:** Correctly predict the average imaginary power introduced to the power spectrum estimate when using time-offset visibilities to estimate $P(k)$.
- **Pipelines Tested:** `hera_pspec.pspecdata`?
- **Criteria**:
  1. Imaginary component of ensemble average computed with `hera_pspec.pspecdata` agrees with semi-analytic expectation to ??? %
  

### Summary

The results of this validation test, in reference to the outlined criteria, are
1. {Test results for criteria 1}
2. {Test results for criteria 2...}

{Brief notes on anything else interesting that was noted during testing}

### Software

HERA software used in this validation test, with associate git commit hash:

* ``pyuvdata``: ``d1829efacb60da384f64a8f25a280441bfa9d68a``
* ``hera_pspec``: ``aca65860d5624356607874d82e3ad1290e6d09fa``

Versions of other software used in this validation test:

* ``aipy``: ``v3.0.0rc2``
* ``healpy``: ``v1.12.9``
* ``multiprocessing``: ``v0.70a1`` (assumed; not documented in Python 3 implementation)
* ``matplotlib``: ``v3.1.0``
* ``numpy``: ``v1.16.4``
* ``re``: ``v2.2.1``

### Data

The following paths reflect the exact locations of all data used in this test:

In [ ]:
path0 = "/lustre/aoc/projects/hera/alanman/eor_sky_sim/"
path1 = "/lustre/aoc/projects/hera/Validation/test-0.1.0/Spectra/"

## Power Spectrum Retrieval from Visibility Simulations

In this section, we show how the power spectra were obtained for this variation of step 0.1. The details are mostly the same as in step-0.1.0, with the slight modification of splitting the visibility data for each simulation into two sets of visibility data: one set corresponds to all the even-indexed time values, whereas the other corresponds to all the odd-indexed time values.

In [ ]:
# try to make a glob of the pre-processed files, make glob of sim files if not
try:
    dfiles = sorted(glob.glob('{}eor*.npz'.format(path1)))
    data = np.load(dfiles[0])['arr_1']
    # XXX do we want to look at the distribution of imaginary powers? can we predict what it will be?
    print('Preprocessed files found. Now histogramming powers and computing ensemble averaged spectrum.')
    prepro = True
except:
    dfiles = sorted(glob.glob('{}*24.*.uvh5'.format(path0)))
    print('Preprocessed files not found. Now beginning power spectrum extraction.')
    print('WARNING: This process may take up to 30 hours. To speed up the process, ensure \
           \nthat your interactive job has requested multiple processors per node and sufficient \
           \nmemory. We recommend at least 5 processors and 64 GB memory.')
    prepro = False

In [ ]:
# either way, we want to make a histogram, so let's define the bin edges
bins = np.linspace(0, 1e5, 49)

# if pre-processed files can't be found, then we need to process the simulation files
if not prepro:
    # if we can't find the pre-processed files, then they must not be on the cluster
    # so let's just process all the files and save them
    
    # define a function that accepts an input uvh5 file, calculates the power spectrum,
    # and saves the result in a PSpecContainer object
    def calc_pspec(infile):
        # use regular expressions to get simulation number
        p1 = re.compile('\d+.uvh5')
        substr = p1.findall(infile)[0]
        p2 = re.compile('\d+')
        fnum = p2.findall(substr)[0]
        # make name for PSpecContainer save file
        outfile = '{}eorsky_offset_spec{}'.format(path1,fnum)
        
        # initialize UVData object and load infile
        uvd = UVData()
        # read data file, but suppress message about eorsky not being a known telescope
        with io.capture_output() as captured:
            uvd.read_uvh5(infile)
        
        # define the cosmology
        cosmo = hp.conversions.Cosmo_Conversions()
        
        # get parameters for making a Gaussian beam
        bm_fwhm = np.radians(uvd.extra_keywords[u'bm_fwhm'])
        freqs = np.unique(uvd.freq_array)
        
        # make the beam
        beam = hp.PSpecBeamGauss(bm_fwhm, freqs, cosmo=cosmo)
        
        # make blpairs using only autopairs
        bls = uvd.get_antpairs()
        blpairs = zip(bls, bls)
            
        # define tapers to be used
        tapers = ['blackman-harris', 'none']
        
        # make sure output is in temperature units
        Jy2mK = uvd.vis_units.upper()=='JY'
        
        # split visibilities according to even/odd time indices
        times = np.unique(np.array(uvd.time_array))
        uvd1 = uvd.select(times=times[:-1:2], inplace=False)
        uvd2 = uvd.select(times=times[1::2], inplace=False)
        
        # calculate the spectra with pspec_run for each taper
        for taper in tapers:
            # update outfile name
            outfile = '{}_{}.psc'.format(outfile, taper)
            
            # capture the output from when Jy2mK is asked to act on a dset already set
            # to mK units
            with io.capture_output() as captured:
                # get power spectrum
                psc, ds = hp.pspecdata.pspec_run([uvd1,uvd2], outfile, blpairs=blpairs,
                                                 taper=taper, beam=beam, cosmo=cosmo,
                                                 Jy2mK=Jy2mK, verbose=False)
        return 0
    
    # now process the files in parallel
    if __name__=='__main__':
        # get the user name to retrieve number of processors requested for job
        user = !whoami
        user = user[0]
        
        # assume everyone follows the guidelines for using jupyter notebooks
        # and assume that an interactive job using multiple processors is run
        # using the job scheduler
        try:
            jn_pbs_path = '/users/{}/jupyter_notebook.pbs'.format(user)
        except:
            print('Please set up a jupyter_notebook.pbs script in your personal directory.')
            sys.exit()
        with open(jn_pbs_path) as f:
            # read the contents of the .pbs script
            text = f.read()
            
            # use regular expressions to find the number of processors requested
            p1 = re.compile('nodes=\d+')
            p2 = re.compile('ppn=\d+')
            p3 = re.compile('\d+')
            
            nodes = int(p3.findall(p1.findall(text)[0])[0])
            ppn = int(p3.findall(p2.findall(text)[0])[0])
            
            # check that the interactive job has enough memory requested, since
            # each instance of an opened uvh5 file via UVData requires approx
            # 2 GB of memory
            p1 = re.compile('vmem=\d+G')
            mem = int(p3.findall(p1.findall(text)[0])[0])

            
        # choose number of processors such that there won't be a MemoryError,
        # but also such that at least one processor is used
        Nprocs = max(1,min(mem/4 - nodes, nodes*(ppn - 1)))
        # note that there are times in the pspec pipeline where the data is copied
        # before it is discarded; this can cause memory spikes up to Nprocs * 4 GB
        # when the pspec pipeline is run in parallel over Nprocs processors

        # start pool of workers
        pool = mp.Pool(Nprocs)
        # process files in parallel
        pool.map(calc_pspec, dfiles)
        # close the pool
        pool.close()
        
    # now that the files have been processed, let's retrieve the ensemble averaged pspec
    
    # first, define some things for some clarity on what's happening
    # first, load in one of the simulation files
    uvd = UVData()
    uvd.read_uvh5(dfiles[np.random.randint(low=0,high=len(dfiles))])
    
    # next, define the tapers used in this analysis
    tapers = ['blackman-harris', 'none']
    
    # now get the baseline pairs; these are (0,11), (0,12), (11,12)
    bls = uvd.get_antpairs()
    blpairs = zip(bls,bls)
    
    # get the size of the time array
    Ntimes = len(np.unique(uvd.time_array))/2 # visibilities were split in half by time
    
    # get the size of the frequency array
    Nfreqs = len(np.unique(uvd.freq_array))
    
    # initialize an array to hold the averaged spectrum
    avg_spec = np.zeros( (len(tapers), len(blpairs), Ntimes, Nfreqs), dtype='complex128' )
    
    # also initialize an array for the histograms
    hist = np.zeros( (len(tapers), len(blpairs), len(bins)-1), dtype='float64' )
    
    # loop over tapers
    for i, taper in enumerate(tapers):
        # make glob of data files
        specfiles = sorted(glob.glob('{}*offset*{}.psc'.format(path1, taper)))
        
        # now loop over files
        for specfile in specfiles:
            # load PSpecContainer object
            psc = hp.container.PSpecContainer(specfile)
            
            # get UVPSpec object from pspec container
            uvp = psc.get_pspec(psc.groups()[0])[0]
            
            # loop over baseline pairs
            for j, blp in enumerate(blpairs):
                # make key to get pspec
                # we used the full spectral range, so spw = 0
                # all of the data has polarization pI
                key = (0, blp, 'pI')
                
                # retrieve the power spectrum
                pspec = uvp.get_data(key)
                
                # add it to the ensemble average array
                avg_spec[i,j] += pspec
                
                # update the histogram
                hist[i,j] += np.histogram(pspec.real, bins=bins)[0]
                
                # normalize the histogram
                hist[i,j] /= hist[i,j].sum()
                            
    # divide out by the number of files for the ensemble average
    avg_spec /= len(dfiles)
    
else:
    # get the ensemble average and fill out the histogram
    
    # first, make an array of zeros for the ensemble average
    avg_spec = np.zeros(data.shape, dtype='complex128')
    
    # since we were able to load in the pre-processed files, let's make
    # histograms for spectra computed with and without a taper
    hist = np.zeros( (2, 3,len(bins)-1), dtype='float64')
    
    # we'll need to loop over data files
    for dfile in dfiles:
        # load in the data
        data = np.load(dfile)['arr_1']
        
        # add it to the average spectrum, to be divided by the number of realizations
        # later on in the script
        avg_spec += data
        
        # update the histograms, but do this on a per-baseline basis
        # we need to take the abs of the data first for histogramming
        data = np.abs(data)
        # from the README, axis-1 corresponds to baselines
        for j in range(data.shape[1]):
            # axis-0 corresponds to taper choice
            # data[0] -> bh-taper
            # data[1] -> no taper
            hist[0,j] += np.histogram(data[1,j], bins=bins)[0]
            hist[1,j] += np.histogram(data[0,j], bins=bins)[0]
            
    # we're done with our loops, so now divide the average spectrum array by the 
    # number of realizations
    avg_spec /= len(dfiles)
    
    # let's normalize the histograms while we're at it
    for j in range(hist.shape[1]):
        hist[0,j] /= hist[0,j].sum()
        hist[1,j] /= hist[1,j].sum()

## Prediction for Imaginary Power

In this section, we outline the model used to predict the existence of imaginary power in the spectra computed in the previous section and present the computational methods used to make the prediction.

### The Model

We begin with our assumptions of the sky used to generate the visibility data used in the previous section. We assume that the signal is sky-locked and that the brightness temperature on the sky is normally distributed in frequency and angular position, and that the value of the brightness temperature in any voxel is uncorrelated with that in any other voxel. We additionally assume that the telescope beam is Gaussian, centered on zenith as determined by the (time-dependent) HERA position.  
  
In order to obtain a prediction for what we expect the `hera_pspec.pspecdata` pipeline to output, we begin with the power spectrum estimator given in Equation 4 of Parsons et al. (ApJ 788 (2), 106, 2014):  
  
$$
\begin{equation}
\hat{P}(\mathbf{k}) = \biggl(\frac{\lambda^2}{2k_b}\biggr)^2\frac{X^2Y}{\Omega_\mathrm{pp}B} \Bigl\langle\tilde{V}_i(\tau,t)\tilde{V}^*_j(\tau,t)\Bigr\rangle_{i<j}, 
\end{equation}
$$
  
where $\lambda$ is the observing frequency, $X$ and $Y$ are cosmological scalars that convert angles and frequencies into comoving distances, $\Omega_\mathrm{pp}$ is the integral of the beam-power-squared over the sky, $B$ is the observing bandwidth, $\langle\cdots\rangle_{i<j}$ denotes an average over redundant baselines, and $\tilde{V}$ is the delay-transformed visibility. We use the usual definition of visibility in a sky-locked frame,  
  
$$
\begin{equation}
V(\nu,t) = \int \mathrm{d}\Omega A_\nu(\hat{\mathbf{n}},t)I_\nu(\hat{\mathbf{n}})\mathrm{e}^{-i2\pi\nu\mathbf{b}(t)\cdot\hat{\mathbf{n}}/c},
\end{equation}
$$
  
where $A$ is the beam-power, $I$ is the intensity on the sky, and $\mathbf{b}$ is the baseline making the visibility measurement. We define the brightness temperature $T_\nu$ such that
  
$$
\begin{equation}
I_\nu(\hat{\mathbf{n}}) = \frac{2k_b}{\lambda^2}T_\nu(\hat{\mathbf{n}}).
\end{equation}
$$
  
The delay-transformed visibility is obtained by taking the Fourier transform of the visibility along the frequency-axis:
  
$$
\begin{equation}
\tilde{V}(\tau,t) =  \frac{2k_b}{\lambda^2}\int \mathrm{d}\nu\mathrm{d}\Omega A_\nu(\hat{\mathbf{n}},t)T_\nu(\hat{\mathbf{n}})\mathrm{e}^{-i2\pi\nu\bigl(\mathbf{b}(t)\cdot\hat{\mathbf{n}}/c - \tau\bigr)}.
\end{equation}
$$
  
Using these definitions, we can rewrite the power spectrum estimator as  
  
$$
\begin{equation}
\hat{P}(\mathbf{k}) = \frac{X^2Y}{\Omega_\mathrm{pp}B}\biggl\langle\int\mathrm{d}\nu'\mathrm{d}\nu\mathrm{d}\Omega'\mathrm{d}\Omega A_{\nu'}(\hat{\mathbf{n}}',t)A_\nu(\hat{\mathbf{n}},t)T_{\nu'}(\hat{\mathbf{n}}')T_\nu(\hat{\mathbf{n}})\exp\bigl(-i2\pi(\nu'-\nu)\tau\bigr)\exp\bigl(-i2\pi(\nu'\mathbf{b}_j\cdot\hat{\mathbf{n}}'/c - \nu\mathbf{b}_i\cdot\hat{\mathbf{n}}/c)\bigr)\biggr\rangle.
\end{equation}
$$
  
(Complex conjugation has been left off of quantities assumed to be real-valued.) We now use the assumption that the temperature field is normally distributed and that the value it takes on in one voxel is independent from that of any other voxel, and take the average over redundant baselines to be an ensemble average, so that  
  
$$
\begin{equation}
\bigl\langle T_{\nu'}(\hat{\mathbf{n}}')T_\nu(\hat{\mathbf{n}})\bigr\rangle = \sigma^2\Delta\nu\Delta\Omega\delta_D(\nu'-\nu)\delta_D(\hat{\mathbf{n}}'-\hat{\mathbf{n}}),
\end{equation}
$$
  
where $\delta_D(\cdots)$ indicates the Dirac delta function, $\sigma^2$ is the variance of the temperature field, $\Delta\nu$ is the size of a frequency channel, and $\Delta\Omega$ is the solid angle subtended by a pixel (for our case, these must be the same value as those used for the simulation). From this, it follows that the power spectrum estimator simplifies to  
  
$$
\begin{equation}
\hat{P}(\mathbf{k}) = \sigma^2\Delta\nu\Delta\Omega\frac{X^2Y}{\Omega_\mathrm{pp}B}\int\mathrm{d}\nu\mathrm{d}\Omega |A_\nu(\hat{\mathbf{n}},t)|^2\exp\bigl(-i2\pi\nu\Delta \mathbf{b}_{ij}(t)\cdot\hat{\mathbf{n}}/c\bigr),
\end{equation}
$$
  
where $\Delta\mathbf{b}_{ij} = \mathbf{b}_j-\mathbf{b}_i$. We can now evaluate the frequency integral over the bandwidth $\nu\in[\nu_1,\nu_2]$ where $\nu_2 - \nu_1 = B$ to get  
  
$$
\begin{equation}
\hat{P}(\mathbf{k}) = \sigma^2\Delta\nu\Delta\Omega\frac{X^2Y}{\Omega_\mathrm{pp}B}\int\mathrm{d}\Omega|A_\nu(\hat{\mathbf{n}}, t)|^2\frac{\mathrm{e}^{-i2\pi\nu_2\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c} - \mathrm{e}^{-i2\pi\nu_1\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c}}{-i2\pi\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c}.
\end{equation}
$$
  
To put the above expression into a nicer form, let's define the band-center frequency $\nu_0 = (\nu_1 + \nu_2)/2$, so that $\nu_2 = \nu_0 + B/2$ and $\nu_1 = \nu_0 - B/2$. We can then factor out $\exp(-i2\pi\nu_0\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c)$ from the difference of exponentials and use Euler's identity, along with the identity $z - z^* = 2i\mathrm{Im}(z)$ for any complex $z$, to rewrite the above equation as  
  
$$
\begin{equation}
\hat{P}(\mathbf{k}) = \sigma^2\Delta\nu\Delta\Omega\frac{X^2Y}{\Omega_\mathrm{pp}B}\int\mathrm{d}\Omega|A_\nu(\hat{\mathbf{n}})|^2\exp\bigl(-i2\pi\nu_0\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c\bigr)\frac{\sin(\pi B\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c)}{\pi\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c}.
\end{equation}
$$
  
We can pull the factor of $1/B$ under the integral and use the definition $\mathrm{sinc}(x) \equiv \sin(x)/x$ to rewrite the above equation somewhat more compactly:  
  
$$
\begin{equation}
\label{eq:pspec-expectation}
\hat{P}(\mathbf{k}) = \sigma^2\Delta\nu\Delta\Omega\frac{X^2Y}{\Omega_\mathrm{pp}}\int\mathrm{d}\Omega|A(\hat{\mathbf{n}}, t)|^2\exp\bigl(-i2\pi\nu_0\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c\bigr)\mathrm{sinc}\bigl(\pi B\Delta\mathbf{b}_{ij}\cdot\hat{\mathbf{n}}/c\bigr).
\end{equation}
$$
  
Notice that in the limit $\Delta\mathbf{b}_{ij} = 0$, the above equation reduces to the expected result:  
  
$$
\begin{equation}
\hat{P}(\mathbf{k}) = \sigma^2\Delta\nu\Delta\Omega X^2Y.
\end{equation}
$$
  
For the case of calculating $\hat{P}(\mathbf{k})$ with autocorrelations of time-offset visibilites, the quantity $\Delta\mathbf{b}_{ij}$ may be written as  
  
$$
\begin{equation}
\Delta\mathbf{b}_{ii}(t) = \bigl(\mathbf{I} - \mathbf{R}(\mathbf{\omega}_\oplus\delta t)\bigr)\mathbf{b}_{i}(t),
\end{equation}
$$
  
where $\mathbf{I}$ is the identity matrix, $\mathbf{R}(\mathbf{\omega}_\oplus\delta t)$ is a matrix that rotates $\mathbf{b}_i$ about the Earth's rotational axis by a phase $\omega_\oplus{\delta}t$, where $\omega_\oplus$ is the angular frequency of Earth's rotation and ${\delta}t$ is the time-offset between visibilities. For our case, ${\delta}t$ corresponds to the integration time-step used in the simulation. In order to determine the expected amplitude of the imaginary component of the power spectrum, we integrate equation$~\ref{eq:pspec-expectation}$ numerically using a HEALPix map.